In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from prefect.blocks.system import Secret
from catnip.fla_redshift import FLA_Redshift
from typing import Dict
from concurrent.futures import ThreadPoolExecutor
import openpyxl 
import calendar

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [ ]:
# 23-24

In [3]:
q = """
select 
    full_opponent, max(original_six_plus_extra)
from 
    custom.cth_game_descriptions
group by 
    full_opponent
order by 
    full_opponent"""

tier_df = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)
tier_df.at[29,'max'] = 0.00

In [4]:
df = pd.read_excel("C:\\Users\\riffere\\Desktop\\new_budget_by_game_2324.xlsx", sheet_name = 'final')

# game day info
# df['start_time_num'] = df.apply(lambda row: 1 if datetime.strptime(row['start_time'][:-3], "%H:%M") <= datetime.strptime('5:00', "%H:%M") 
#                     or datetime.strptime(row['start_time'][:-3], "%H:%M") == datetime.strptime('12:30', "%H:%M") else 2, axis = 1)
# df['is_early'] = df.apply(lambda row: 1 if row['start_time_num'] == 1 else 0, axis = 1)
# df['is_evening'] = df.apply(lambda row: 1 if row['start_time_num'] == 2 else 0, axis = 1)

df['event_date'] = pd.to_datetime(df['event_date']).dt.date
df['week_day'] = df.apply(lambda row: 1 if datetime.strftime(row['event_date'], "%w") == '6' else (
    2 if datetime.strftime(row['event_date'], "%w") == '5' or datetime.strftime(row['event_date'], "%w") == '0' else 3), axis = 1)
df['is_fri_sun'] = df.apply(lambda row: 1 if row['week_day'] == 2 else 0, axis = 1)
df['is_sat'] = df.apply(lambda row: 1 if row['week_day'] == 1 else 0, axis = 1)
df['is_weekday'] = df.apply(lambda row: 1 if row['week_day'] == 3 else 0, axis = 1)

df['trimester'] = df.apply(lambda row: 1 if 9 <= row['event_date'].month < 12 else 
                    (2 if row['event_date'].month < 2 or row['event_date'].month == 12 else 3), axis = 1)
df['is_trimester_1'] = df.apply(lambda row: 1 if row['trimester'] == 1 else 0, axis = 1)
df['is_trimester_2'] = df.apply(lambda row: 1 if row['trimester'] == 2 else 0, axis = 1)
df['is_trimester_3'] = df.apply(lambda row: 1 if row['trimester'] == 3 else 0, axis = 1)

df['event_date_date'] = pd.to_datetime(df['event_date']).dt.date
df['week_day'] = df.apply(lambda row: 1 if datetime.strftime(row['event_date'], "%w") == '6' else (
    2 if datetime.strftime(row['event_date'], "%w") == '5' or datetime.strftime(row['event_date'], "%w") == '0' else 3), axis = 1)
df['trimester'] = df.apply(lambda row: 1 if 9 <= row['event_date'].month < 12 else 
                    (2 if row['event_date'].month < 2 or row['event_date'].month == 12 else 3), axis = 1)

thanksgiving = {2017 : 23, 2018 : 22, 2019 : 28, 2020 : 26, 2021: 25, 2022: 24, 2023: 23, 2024: 28, 2025: 27, 2026: 26}
df['is_thanksgiving'] = df.apply(lambda row: 1 if row['event_date'].month == 11 and 
                (thanksgiving[row['event_date'].year]-3) <= row['event_date'].day <= (thanksgiving[row['event_date'].year]+3) else 0, axis = 1)
df['is_christmas'] = df.apply(lambda row: 1 if row['event_date'].month == 12 and 21 <= row['event_date'].day <= 27 else 0, axis = 1)
df['is_nye'] = df.apply(lambda row: 1 if row['event_date'].month == 12 and 29 <= row['event_date'].day <= 31 else 
                (1 if row['event_date'].month == 1 and 1 <= row['event_date'].day <= 4 else 0), axis = 1)
df['is_holiday'] = df.apply(lambda row: 1 if row['is_thanksgiving'] == 1 or row['is_christmas'] == 1 or 
    row['is_nye'] == 1 else 0, axis = 1)

# density
calendar_df = pd.DataFrame(columns = ['days_of_the_year'], data = 
    ['01-01','01-02','01-03','01-04','01-05','01-06','01-07','01-08','01-09','01-10','01-11',
    '01-12','01-13','01-14','01-15','01-16','01-17','01-18','01-19','01-20','01-21','01-22',
    '01-23','01-24','01-25','01-26','01-27','01-28','01-29','01-30','01-31',
    '02-01','02-02','02-03','02-04','02-05','02-06','02-07','02-08','02-09','02-10','02-11',
    '02-12','02-13','02-14','02-15','02-16','02-17','02-18','02-19','02-20','02-21','02-22',
    '02-23','02-24','02-25','02-26','02-27','02-28',
    '03-01','03-02','03-03','03-04','03-05','03-06','03-07','03-08','03-09','03-10','03-11',
    '03-12','03-13','03-14','03-15','03-16','03-17','03-18','03-19','03-20','03-21','03-22',
    '03-23','03-24','03-25','03-26','03-27','03-28','03-29','03-30','03-31',
    '04-01','04-02','04-03','04-04','04-05','04-06','04-07','04-08','04-09','04-10','04-11',
    '04-12','04-13','04-14','04-15','04-16','04-17','04-18','04-19','04-20','04-21','04-22',
    '04-23','04-24','04-25','04-26','04-27','04-28','04-29','04-30',
    '05-01','05-02','05-03','05-04','05-05','05-06','05-07','05-08','05-09','05-10','05-11',
    '05-12','05-13','05-14','05-15','05-16','05-17','05-18','05-19','05-20','05-21','05-22',
    '05-23','05-24','05-25','05-26','05-27','05-28','05-29','05-30','05-31',
    '06-01','06-02','06-03','06-04','06-05','06-06','06-07','06-08','06-09','06-10','06-11',
    '06-12','06-13','06-14','06-15','06-16','06-17','06-18','06-19','06-20','06-21','06-22',
    '06-23','06-24','06-25','06-26','06-27','06-28','06-29','06-30',
    '07-01','07-02','07-03','07-04','07-05','07-06','07-07','07-08','07-09','07-10','07-11',
    '07-12','07-13','07-14','07-15','07-16','07-17','07-18','07-19','07-20','07-21','07-22',
    '07-23','07-24','07-25','07-26','07-27','07-28','07-29','07-30','07-31',
    '08-01','08-02','08-03','08-04','08-05','08-06','08-07','08-08','08-09','08-10','08-11',
    '08-12','08-13','08-14','08-15','08-16','08-17','08-18','08-19','08-20','08-21','08-22',
    '08-23','08-24','08-25','08-26','08-27','08-28','08-29','08-30','08-31',
    '09-01','09-02','09-03','09-04','09-05','09-06','09-07','09-08','09-09','09-10','09-11',
    '09-12','09-13','09-14','09-15','09-16','09-17','09-18','09-19','09-20','09-21','09-22',
    '09-23','09-24','09-25','09-26','09-27','09-28','09-29','09-30',
    '10-01','10-02','10-03','10-04','10-05','10-06','10-07','10-08','10-09','10-10','10-11',
    '10-12','10-13','10-14','10-15','10-16','10-17','10-18','10-19','10-20','10-21','10-22',
    '10-23','10-24','10-25','10-26','10-27','10-28','10-29','10-30','10-31',
    '11-01','11-02','11-03','11-04','11-05','11-06','11-07','11-08','11-09','11-10','11-11',
    '11-12','11-13','11-14','11-15','11-16','11-17','11-18','11-19','11-20','11-21','11-22',
    '11-23','11-24','11-25','11-26','11-27','11-28','11-29','11-30',
    '12-01','12-02','12-03','12-04','12-05','12-06','12-07','12-08','12-09','12-10','12-11',
    '12-12','12-13','12-14','12-15','12-16','12-17','12-18','12-19','12-20','12-21','12-22',
    '12-23','12-24','12-25','12-26','12-27','12-28','12-29','12-30','12-31'])
event_dates = df['event_date_date'].values
def get_range(date : datetime.date, calendar_df_active):
    month = date.month
    day = date.day
    year = date.year
    if calendar.isleap(year):
        leap_day = pd.DataFrame(columns = ['days_of_the_year'], data = ['02-29'])
        calendar_df_active = pd.concat([calendar_df_active[0:59], leap_day, calendar_df_active[59:]], ignore_index= True)
    if len(str(month)) == 1:
        if len(str(day)) == 1:
            date = str('0' + str(month) + '-0' + str(day))
        else:
            date = str('0' + str(month) + '-' + str(day))
    else:
        if len(str(day)) == 1:
            date = str(str(month) + '-0' + str(day))
        else:
            date = str(str(month) + '-' + str(day))
    if date < '01-04':
        a = calendar_df_active[:calendar_df_active.index[calendar_df_active['days_of_the_year'] == date].tolist()[0]+4]
        left = 7 - len(a)
        b = calendar_df_active[(365-left):].values
        total = np.concatenate((a,b), axis = 0)
    elif date > '12-28':
        b = calendar_df_active[calendar_df_active.index[calendar_df_active['days_of_the_year'] == date].tolist()[0]-3:]
        left = 7 - len(b)
        a = calendar_df_active[0:(left)].values
        total = np.concatenate((b,a), axis = 0)
    else:
        total = calendar_df_active[calendar_df_active.index[calendar_df_active['days_of_the_year'] == date].tolist()[0]-3:
                calendar_df_active.index[calendar_df_active['days_of_the_year'] == date].tolist()[0]+4].values
    val = 0
    for i in total:
        month = i[0][0:2]
        day = i[0][3:]
        date = str(str(year) + '-' + month + '-' + day)
        current_date = datetime.strptime(date, '%Y-%m-%d').date()
        if current_date in event_dates:
            val += 1
    return val
df['density'] = df.apply(lambda row: get_range(row['event_date'], calendar_df),axis = 1)
df['is_dense'] = df.apply(lambda row: 1 if row['density'] >= 3 else 0, axis = 1)
df['is_not_dense'] = df.apply(lambda row: 1 if row['density'] < 3 else 0, axis = 1)

df.at[0,'is_holiday'] = 1
df = df.rename({'Away': 'full_opponent'}, axis = 1)
df = df.merge(tier_df, on = 'full_opponent', how = 'left')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\riffere\\Desktop\\new_budget_by_game_2324.xlsx'

In [51]:
teams = {'Boston Bruins' : [1, 1, 1, 1.5], 'Buffalo Sabres' : [1, 1, 0, 0.889], 'Detroit Red Wings' : [1, 1, 0.75, 1.2], 'Florida Panthers' : [1, 1, 0, 0.426],
         'Montreal Canadiens' : [1, 1, 0.75, 1.6], 'Ottawa Senators' : [1, 1, 0, 0.608], 'Tampa Bay Lightning' : [1, 1, 1.5, 0.807], 'Toronto Maple Leafs' : [1, 1, 1, 2],
        'Carolina Hurricanes' : [1, 2, 0, 0.543], 'Columbus Blue Jackets' : [1, 2, 0, 0.514], 'New Jersey Devils' : [1, 2, 0, 0.755], 'New York Islanders' : [1, 2, 0, 0.587], 
        'New York Rangers' : [1, 2, 1, 1.4], 'Philadelphia Flyers' : [1, 2, 1, 1.5], 'Pittsburgh Penguins' : [1, 2, 0.5, 1.8], 'Washington Capitals' : [1, 2, 0.75, 0.809],
        'Arizona Coyotes' : [0, 3, 0, 0.399], 'Chicago Blackhawks' : [0, 3, 1, 2.4], 'Colorado Avalanche' : [0, 3, 0, 0.634], 'Dallas Stars' : [0, 3, 0, 1.1], 
        'Minnesota Wild' : [0, 3, 0, 0.739], 'Nashville Predators' : [0, 3, 0, 0.638], 'St. Louis Blues' : [00, 3, 0, 0.799], 'Winnipeg Jets' : [0, 3, 0, 0.581],
        'Anaheim Ducks' : [0, 4, 0, 0.611], 'Calgary Flames' : [0, 4, 0, 0.675], 'Edmonton Oilers' : [0, 4, 0, 0.908], 'Los Angeles Kings' : [0, 4, 0, 1.1],
         'San Jose Sharks' : [0, 4, 0, 0.817], 'Seattle Kraken' : [0, 4, 0, 0.322], 'Vancouver Canucks' : [0, 4, 0, 1], 'Vegas Golden Knights' : [0, 4, 0.75, 0.520]}

df['conf'] = df.apply(lambda row: teams[row['full_opponent']][0], axis = 1)
df['div'] = df.apply(lambda row: teams[row['full_opponent']][1], axis = 1)
df['original_six_plus_extra'] = df.apply(lambda row: teams[row['full_opponent']][2], axis = 1)
df['twitter_followers'] = df.apply(lambda row: teams[row['full_opponent']][3], axis = 1)

    conf  div  original_six_plus_extra  twitter_followers
0      1    1                     1.50              0.807
1      1    1                     1.00              2.000
2      0    4                     0.00              1.000
3      0    4                     0.00              0.817
4      0    4                     0.00              0.322
5      1    2                     0.00              0.514
6      1    2                     0.00              0.543
7      0    3                     1.00              2.400
8      0    4                     0.00              0.908
9      1    1                     1.00              1.500
10     0    3                     0.00              0.581
11     1    2                     0.00              0.587
12     0    3                     0.00              1.100
13     1    2                     0.50              1.800
14     0    3                     0.00              0.799
15     0    4                     0.75              0.520
16     1    2 

In [ ]:
# 2024-25

In [90]:
q = """
with historical_games as
    (select
         distinct full_opponent, reverse(split_part(reverse(full_opponent), ' ', 1)) as team, abbreviation
    from
        custom.cth_game_descriptions),
seatgeek_table as
    (SELECT
    products.event_date as event_datetime, products.product_description, reverse(split_part(reverse(product_description), ' ', 1)) as team
FROM
    custom.seatgeek_v_products products
INNER JOIN
    custom.seatgeek_v_products_details details ON products.product_key = details.product_key
        AND details.variable = 'showtype' AND details.value IN ('2024-25 Panthers Hockey')
ORDER BY
    products.event_date DESC)
select
    historical_games.*, event_datetime, product_description
from
    seatgeek_table
left join
    historical_games on seatgeek_table.team = historical_games.team
order by
    event_datetime
    """

df = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)

df.at[6,'full_opponent'] = 'Dallas Stars'
df.at[19,'full_opponent'] = 'Tampa Bay Lightning'
df.at[38,'full_opponent'] = 'Utah Hockey Club'

df.at[6,'abbreviation'] = 'DAL'
df.at[19,'abbreviation'] = 'TBL'
df.at[38,'abbreviation'] = 'UTA'

In [91]:
df['event_date'] = pd.to_datetime(df['event_datetime']).dt.date
df['event_datetime_utc'] = df['event_datetime'].dt.tz_localize('US/Eastern')
df['event_datetime_utc'] = df['event_datetime_utc'].dt.tz_convert('UTC')
df['week_day'] = df.apply(lambda row: 1 if datetime.strftime(row['event_date'], "%w") == '6' else (
     2 if datetime.strftime(row['event_date'], "%w") == '5' or datetime.strftime(row['event_date'], "%w") == '0' else 3), axis = 1)
df['day_of_week'] = df['event_datetime'].dt.strftime('%a')
df['start_time'] = df['event_datetime'].dt.strftime('%I:%M%p')
df['start_time_tableau'] = df['event_datetime'].dt.strftime('%I:%M%p')
df['season'] = '2024-25'

df['trimester'] = df.apply(lambda row: 1 if 9 <= row['event_date'].month < 12 else 
                     (2 if row['event_date'].month < 2 or row['event_date'].month == 12 else 3), axis = 1)

thanksgiving = {2017 : 23, 2018 : 22, 2019 : 28, 2020 : 26, 2021: 25, 2022: 24, 2023: 23, 2024: 28, 2025: 27, 2026: 26}
df['is_thanksgiving'] = df.apply(lambda row: 1 if row['event_date'].month == 11 and 
                (thanksgiving[row['event_date'].year]-3) <= row['event_date'].day <= (thanksgiving[row['event_date'].year]+3) else 0, axis = 1)
df['is_christmas'] = df.apply(lambda row: 1 if row['event_date'].month == 12 and 21 <= row['event_date'].day <= 27 else 0, axis = 1)
df['is_nye'] = df.apply(lambda row: 1 if row['event_date'].month == 12 and 29 <= row['event_date'].day <= 31 else 
                (1 if row['event_date'].month == 1 and 1 <= row['event_date'].day <= 4 else 0), axis = 1)
df['is_holiday'] = df.apply(lambda row: 1 if row['is_thanksgiving'] == 1 or row['is_christmas'] == 1 or 
    row['is_nye'] == 1 else 0, axis = 1)

df.at[2,'is_holiday'] = 1

In [92]:
# density
calendar_df = pd.DataFrame(columns = ['days_of_the_year'], data = 
    ['01-01','01-02','01-03','01-04','01-05','01-06','01-07','01-08','01-09','01-10','01-11',
    '01-12','01-13','01-14','01-15','01-16','01-17','01-18','01-19','01-20','01-21','01-22',
    '01-23','01-24','01-25','01-26','01-27','01-28','01-29','01-30','01-31',
    '02-01','02-02','02-03','02-04','02-05','02-06','02-07','02-08','02-09','02-10','02-11',
    '02-12','02-13','02-14','02-15','02-16','02-17','02-18','02-19','02-20','02-21','02-22',
    '02-23','02-24','02-25','02-26','02-27','02-28',
    '03-01','03-02','03-03','03-04','03-05','03-06','03-07','03-08','03-09','03-10','03-11',
    '03-12','03-13','03-14','03-15','03-16','03-17','03-18','03-19','03-20','03-21','03-22',
    '03-23','03-24','03-25','03-26','03-27','03-28','03-29','03-30','03-31',
    '04-01','04-02','04-03','04-04','04-05','04-06','04-07','04-08','04-09','04-10','04-11',
    '04-12','04-13','04-14','04-15','04-16','04-17','04-18','04-19','04-20','04-21','04-22',
    '04-23','04-24','04-25','04-26','04-27','04-28','04-29','04-30',
    '05-01','05-02','05-03','05-04','05-05','05-06','05-07','05-08','05-09','05-10','05-11',
    '05-12','05-13','05-14','05-15','05-16','05-17','05-18','05-19','05-20','05-21','05-22',
    '05-23','05-24','05-25','05-26','05-27','05-28','05-29','05-30','05-31',
    '06-01','06-02','06-03','06-04','06-05','06-06','06-07','06-08','06-09','06-10','06-11',
    '06-12','06-13','06-14','06-15','06-16','06-17','06-18','06-19','06-20','06-21','06-22',
    '06-23','06-24','06-25','06-26','06-27','06-28','06-29','06-30',
    '07-01','07-02','07-03','07-04','07-05','07-06','07-07','07-08','07-09','07-10','07-11',
    '07-12','07-13','07-14','07-15','07-16','07-17','07-18','07-19','07-20','07-21','07-22',
    '07-23','07-24','07-25','07-26','07-27','07-28','07-29','07-30','07-31',
    '08-01','08-02','08-03','08-04','08-05','08-06','08-07','08-08','08-09','08-10','08-11',
    '08-12','08-13','08-14','08-15','08-16','08-17','08-18','08-19','08-20','08-21','08-22',
    '08-23','08-24','08-25','08-26','08-27','08-28','08-29','08-30','08-31',
    '09-01','09-02','09-03','09-04','09-05','09-06','09-07','09-08','09-09','09-10','09-11',
    '09-12','09-13','09-14','09-15','09-16','09-17','09-18','09-19','09-20','09-21','09-22',
    '09-23','09-24','09-25','09-26','09-27','09-28','09-29','09-30',
    '10-01','10-02','10-03','10-04','10-05','10-06','10-07','10-08','10-09','10-10','10-11',
    '10-12','10-13','10-14','10-15','10-16','10-17','10-18','10-19','10-20','10-21','10-22',
    '10-23','10-24','10-25','10-26','10-27','10-28','10-29','10-30','10-31',
    '11-01','11-02','11-03','11-04','11-05','11-06','11-07','11-08','11-09','11-10','11-11',
    '11-12','11-13','11-14','11-15','11-16','11-17','11-18','11-19','11-20','11-21','11-22',
    '11-23','11-24','11-25','11-26','11-27','11-28','11-29','11-30',
    '12-01','12-02','12-03','12-04','12-05','12-06','12-07','12-08','12-09','12-10','12-11',
    '12-12','12-13','12-14','12-15','12-16','12-17','12-18','12-19','12-20','12-21','12-22',
    '12-23','12-24','12-25','12-26','12-27','12-28','12-29','12-30','12-31'])

event_dates = df['event_date'].values

def get_range(date : datetime.date, calendar_df_active):
    month = date.month
    day = date.day
    year = date.year
    if calendar.isleap(year):
        leap_day = pd.DataFrame(columns = ['days_of_the_year'], data = ['02-29'])
        calendar_df_active = pd.concat([calendar_df_active[0:59], leap_day, calendar_df_active[59:]], ignore_index= True)
    if len(str(month)) == 1:
        if len(str(day)) == 1:
            date = str('0' + str(month) + '-0' + str(day))
        else:
            date = str('0' + str(month) + '-' + str(day))
    else:
        if len(str(day)) == 1:
            date = str(str(month) + '-0' + str(day))
        else:
            date = str(str(month) + '-' + str(day))
    if date < '01-04':
        a = calendar_df_active[:calendar_df_active.index[calendar_df_active['days_of_the_year'] == date].tolist()[0]+4]
        left = 7 - len(a)
        b = calendar_df_active[(365-left):].values
        total = np.concatenate((a,b), axis = 0)
    elif date > '12-28':
        b = calendar_df_active[calendar_df_active.index[calendar_df_active['days_of_the_year'] == date].tolist()[0]-3:]
        left = 7 - len(b)
        a = calendar_df_active[0:(left)].values
        total = np.concatenate((b,a), axis = 0)
    else:
        total = calendar_df_active[calendar_df_active.index[calendar_df_active['days_of_the_year'] == date].tolist()[0]-3:
                calendar_df_active.index[calendar_df_active['days_of_the_year'] == date].tolist()[0]+4].values
    val = 0
    for i in total:
        month = i[0][0:2]
        day = i[0][3:]
        date = str(str(year) + '-' + month + '-' + day)
        current_date = datetime.strptime(date, '%Y-%m-%d').date()
        if current_date in event_dates:
            val += 1
    return val

df['density'] = df.apply(lambda row: get_range(row['event_date'], calendar_df),axis = 1)
df['is_dense'] = df.apply(lambda row: 1 if row['density'] >= 3 else 0, axis = 1)

df['days_before'] = df['event_date'].diff().dt.days
df['days_after'] = (pd.to_datetime(df['event_date'].shift(-1)) - pd.to_datetime(df['event_date'])).dt.days

In [93]:
tiers = pd.read_csv("C:\\Users\\riffere\\Desktop\\final_tier_2425.csv")

df['event_date'] = pd.to_datetime(df['event_date'])
tiers['event_date'] = pd.to_datetime(tiers['event_date'])

df = pd.merge(df, tiers, how = 'left', on = 'event_date')

In [94]:
df['game_type'] = [-1 if x == 'Pre' else 1 for x in df['tier']]

q = """
select 
    distinct full_opponent, max(original_six_plus_extra) as original_six_plus_extra, max(is_conf) as conf, max(div) as div
from 
    custom.cth_game_descriptions
group by 
    full_opponent
order by 
    full_opponent"""

opponent_info = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)
opponent_info.at[29,'original_six_plus_extra'] = 0.00
opponent_info.at[20,'original_six_plus_extra'] = 0.50
opponent_info.at[1,'full_opponent'] = 'Utah Hockey Club'

df = pd.merge(df, opponent_info, how = 'left', on = 'full_opponent')

In [95]:
df[['event_date', 'season', 'abbreviation', 'full_opponent', 'start_time', 'start_time_tableau', 'event_datetime', 'event_datetime_utc', 
    'day_of_week', 'tier','is_premier','trimester','density','is_dense','is_holiday', 'game_type', 'days_before', 'days_after', 'original_six_plus_extra', 'div', 'conf']]

# theme_night_var, opp_pp_3_seasons, home_rolling_points_percentage, overall_points_percentage, cluster, cluster_season, event_name_parameter, budget_goal, game_number 

,event_date,season,abbreviation,full_opponent,start_time,start_time_tableau,event_datetime,event_datetime_utc,day_of_week,tier,...,trimester,density,is_dense,is_holiday,game_type,days_before,days_after,original_six_plus_extra,div,conf
0,2024-09-28,2024-25,CAR,Carolina Hurricanes,07:00PM,07:00PM,2024-09-28 19:00:00,2024-09-28 23:00:00+00:00,Sat,Pre,...,1,2,0,0,-1,NaN,2.0,0.00,2,1
1,2024-09-30,2024-25,TBL,Tampa Bay Lightning,07:00PM,07:00PM,2024-09-30 19:00:00,2024-09-30 23:00:00+00:00,Mon,Pre,...,1,2,0,0,-1,2.0,8.0,1.50,1,1
2,2024-10-08,2024-25,BOS,Boston Bruins,07:00PM,07:00PM,2024-10-08 19:00:00,2024-10-08 23:00:00+00:00,Tue,B,...,1,1,0,1,1,8.0,9.0,1.00,1,1
3,2024-10-17,2024-25,VAN,Vancouver Canucks,07:00PM,07:00PM,2024-10-17 19:00:00,2024-10-17 23:00:00+00:00,Thu,E,...,1,2,0,0,1,9.0,2.0,0.00,4,0
4,2024-10-19,2024-25,VGK,Vegas Golden Knights,06:00PM,06:00PM,2024-10-19 18:00:00,2024-10-19 22:00:00+00:00,Sat,D,...,1,3,1,0,1,2.0,3.0,0.75,4,1
5,2024-10-22,2024-25,MIN,Minnesota Wild,06:30PM,06:30PM,2024-10-22 18:30:00,2024-10-22 22:30:00+00:00,Tue,E,...,1,2,0,0,1,3.0,11.0,0.00,3,0
6,2024-11-02,2024-25,DAL,Dallas Stars,06:00PM,06:00PM,2024-11-02 18:00:00,2024-11-02 22:00:00+00:00,Sat,F,...,1,1,0,0,1,11.0,5.0,0.00,3,0
7,2024-11-07,2024-25,NSH,Nashville Predators,07:00PM,07:00PM,2024-11-07 19:00:00,2024-11-08 00:00:00+00:00,Thu,E,...,1,2,0,0,1,5.0,0.0,0.00,3,0
8,2024-11-07,2024-25,NAS,Nashville Predators,07:00PM,07:00PM,2024-11-07 19:00:00,2024-11-08 00:00:00+00:00,Thu,E,...,1,2,0,0,1,0.0,2.0,0.00,3,0
9,2024-11-09,2024-25,PHI,Philadelphia Flyers,06:00PM,06:00PM,2024-11-09 18:00:00,2024-11-09 23:00:00+00:00,Sat,C,...,1,3,1,0,1,2.0,3.0,0.50,2,1
